In [1]:
import os
import psycopg
import pymongo
import statistics
import pandas as pd
from tqdm import tqdm
from collections import defaultdict 
import binascii

from tools.utils.utils import get_mongodb_collections, get_one_document_from_mongodb_by_key, _create_token_set, get_initial_spark_rdd

In [ ]:
mc, coll = get_mongodb_collections('gittables', False)

doc = get_one_document_from_mongodb_by_key('_id_numeric', 8, *coll)
pd.DataFrame(doc['content'])

In [ ]:
pd.isna(doc['content'][26][1])

In [ ]:
pd.DataFrame(doc['content']).isna().sum()

In [ ]:
ts = _create_token_set(doc['content'], 'set', doc['numeric_columns'])
sorted(ts)

In [ ]:
'{"$numberDouble": "NaN"}' in ts, "NaN" in ts

In [2]:
spark_jars_packages = [
    'org.postgresql:postgresql:42.7.3',
    'org.mongodb.spark:mongo-spark-connector_2.12:10.3.0'
]

# filtering only those tables that have very few cells (<10)
tables_thresholds = {
    'min_row':      5,
    'min_column':   2,
    'min_area':     0,
    'max_row':      999999,
    'max_column':   999999,
    'max_area':     999999,
}

spark, initial_rdd = get_initial_spark_rdd('gittables', True, 72, tables_thresholds, spark_jars_packages)

mode = 'set'

def prepare_tuple(t):
    global mode
    # t = (_id_numeric, content, numeric_columns)
    if t[0] == 8:
        print(t[1][26][1], type(t[1][26][1]), pd.isna(t[1][26][1]), bool(t[1][26][1]))
    return [t[0], _create_token_set(t[1], mode, t[2])]    

token_sets = (
    initial_rdd
        .map(
            # from MongoDB directly
            # (_id_numeric, content, numeric_columns) -> (_id_numeric, [token1, token2, token3, ...])
            lambda t: prepare_tuple(t)
        )
            .flatMap(
                # (set_id, [tok1, tok2, tok3, ...]) -> [(tok1, set_id), (tok2, set_id), ...]
                lambda t: [ (token, t[0]) for token in t[1]]
            )
)

import mmh3
import numpy as np


posting_lists_sorted = (
    token_sets
        .groupByKey()
            .map(
                # (token, [set_idK, set_idJ, set_idM, ...]) -> (token, [set_id1, set_id2, set_id3, ..., set_idZ]) 
                lambda t: (t[0], sorted(list(t[1]))))
                .map(
                    # (token, set_ids) -> (token, set_ids, set_ids_hash)
                    lambda t:
                        (
                            t[0], 
                            t[1]
                        )
                    )
                    .sortBy(
                        # t: (token, setIDs, hash)
                        lambda t: (len(t[1]), t[1]))                            
).collect()

:: loading settings :: url = jar:file:/data4/nanni/anaconda3/envs/nanni-tesi-env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nanni/.ivy2/cache
The jars for the packages stored in: /home/nanni/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-43a0d198-42cd-4796-98ab-15b891de7387;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
	found org.mongodb.spark#mongo-spark-connector_2.12;10.3.0 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
:: resolution report :: resolve 2004ms :: artifacts dl 8ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.mongodb#bson;4.8.2 from central in [default]
	org.mongo

In [3]:
posting_lists_sorted = {p[0]: p[1] for p in posting_lists_sorted}

In [4]:
posting_lists_sorted['{"$numberDouble": "NaN"}']

KeyError: '{"$numberDouble": "NaN"}'

In [ ]:
str(posting_lists_sorted)[3540079-50:3540079+5]